In [1]:
import pymongo
import logging
import datetime
import logging.handlers

def create_tenant(data, created_by):
    # Connect to MongoDB
    client = pymongo.MongoClient('mongodb://localhost:27017/')
    db = client[data['tenant_name']]
    collection = db['sdwan']

    # Create tenant document
    tenant = {
        'customer_name': data['customer_name'],
        'customer_id': data['customer_id'],
        'customer_email': data['customer_email'],
        'services': data['services'],
        'created_at': datetime.datetime.utcnow(),
        'created_by': created_by
    }

    try:
        # Insert tenant document and get _id field
        result = collection.insert_one(tenant)
        document_id = result.inserted_id

        # Log success message with creation time and user
        logging.info(f"Created tenant with _id {document_id} at {tenant['created_at']} by {created_by}")
    except Exception as e:
        # Log error message with creation time and user
        logging.error(f"Failed to create tenant at {datetime.datetime.utcnow()} by {created_by}: {e}")
        raise e

    # Return _id field
    return str(document_id)

In [ ]:

# Initialize logging
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s', 
    level=logging.INFO
)

# Create a syslog handler and add it to the root logger
syslog_handler = logging.handlers.SysLogHandler(address=('192.168.10.199', 3514))
logging.getLogger().addHandler(syslog_handler)

# Create tenant
data = {
    'tenant_name': 'acme_inc',
    'customer_name': 'Acme Inc',
    'customer_id': '123456',
    'customer_email': 'jdoe@acme.com',
    'services': {
        'sdwan': True,
        'security': False,
        'voice': True
    }
}

created_by = 'user1'

try:
    document_id = create_tenant(data, created_by)
except Exception as e:
    logging.exception("Exception occurred")

# Close logging
logging.shutdown()